# Optimizing token usages

Playing around with different number of aspects within a single prompt

In [1]:
GAME_ASPECTS = [
    'Gameplay', 
    'Narrative', 
    'Accessibility', 
    'Sound', 
    'Graphics & Art Design', 
    'Performance', 
    'Bug', 
    'Suggestion', 
    'Price', 
    'Overall'
]

In [2]:
import reviews._reviews02 as reviews02

In [3]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# select a comment to test with
review_text = reviews02.sample_01

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents([review_text], metadatas=[{"source":"review_01"}])

embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_func)

retriever = db.as_retriever(search_kwargs={"k": 5})

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
from langchain_community.llms import Ollama

llm = Ollama(model='llama2:7b-chat-q4_K_M', temperature=0.2)        # set to a lower temperature for more consistent results

Before our test, a code is added to track token usage for different prompting strategies.

In [5]:
# define callbacks for detecting token usage

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print('Response in callback')
        print(response)
        print()

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

Here we test for different number of aspects within a single prompt.

1. One aspect in a prompt
2. 2 aspects in a prompt -> a (2, 2, 2, 2, 2) prompting combination
3. 3-4 aspects in a prompt -> a (3, 3, 4) prompting combination
4. 5 aspects in a prompt
5. 10 aspects in a prompt

Single aspect

In [6]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect is: '''

aspects_response = {k: '' for k in GAME_ASPECTS}
sum_of_token_usage, sum_of_time_usage = 0, 0

for aspect in GAME_ASPECTS:
    my_question = my_question_template + f'{aspect}'
    print(my_question)

    chain =  RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                template=prompt_template,
                input_variables=["summaries", "question"],
            )
        },
        return_source_documents=True,
    )

    response = chain.invoke(
        {
            'question': my_question
        },
        config=chain_config
    )

    print('\'\'\'')
    print(response['answer'])
    print('\'\'\'')
    aspects_response[aspect] = response['answer']
    print('-'*10)
    print('Sources')
    for doc in response['source_documents']:
        print(doc.page_content)
        print(doc.metadata)
        print('-'*10)
    token_usage_obj = common_deque.pop()
    token_usage, time_usage = token_usage_obj['token_usage'], token_usage_obj['time_costed']
    sum_of_token_usage += token_usage
    sum_of_time_usage += time_usage
    print(f'Token Usage: {token_usage}')

    print('-'*20)

print('Summary of token usage:', sum_of_token_usage)
print('Summary of time usage:', sum_of_time_usage / 1e9, 's')

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect is: Gameplay


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text='\nGameplay:\n\n* NA - No information provided in the review about gameplay.', generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:10.790017469Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 29945, 448, 29871, 29941, 29945, 285, 567, 373, 1716, 4482, 322, 8494, 336, 6055, 29889, 366, 525, 270, 1348, 393, 565, 8494, 336, 471, 29871, 29941, 29900, 448, 29871, 29941, 29945, 769, 4482, 881, 367, 29871, 29953, 29900, 29892, 541, 694, 29889, 1584, 411, 4236, 6055, 3748, 1603, 5148, 7736, 1156, 766, 17961, 701, 448, 21640, 29889, 263, 395, 29871, 29955, 29900, 3611, 881, 1065, 472, 2987

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


...

Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game related to performance that can be extracted from the review:\n\n1. Frame rate: Poorly optimized, runs between 25-35 FPS on both low and ultra settings.\n2. Up-scaling: Disabling up-scaling improves the game's appearance, but it should not be necessary to do so.\n3. Minimum frame rate: A $70 title should run at 50-60 FPS on the lowest settings minimum.\n4. Optimization: Trying to optimize settings to get better frames has resulted in many hours of gameplay and no refund.\n\nHere are the short descriptions for each aspect:\n\n1. Frame rate: Poor performance with inconsistent frame rates.\n2. Up-scaling: Unnecessary compromise in graphics quality.\n3. Minimum frame rate: Underwhelming experience with too much lag.\n4. Optimization: Time-consuming and frustrating optimization process.", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:13.59304945Z', 'response

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text='\nSure! Here are the aspects of the game and their corresponding 20-word descriptions based on the review:\n\n1. Suggestion for optimization: NA\n2. Expected FPS range: 50-60 fps (minimum)\n3. Actual FPS range: 25-35 fps (both low and ultra settings)', generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:14.633958478Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 29945, 448, 29871, 29941, 29945, 285, 567, 373, 1716, 4482, 322, 8494, 336, 6055, 29889, 366, 525, 270, 1348, 393, 565, 8494, 336, 471, 29871, 29941, 29900, 448, 29871, 29941, 29945, 769, 4482, 881, 367, 29871, 29953, 29

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text='\nHere are the aspects of the game related to price and their corresponding descriptions based on the review:\n\nPrice: $70\nDescription: A $70 title should run at 50-60 fps on the lowest settings minimum.', generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:15.195951978Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 29945, 448, 29871, 29941, 29945, 285, 567, 373, 1716, 4482, 322, 8494, 336, 6055, 29889, 366, 525, 270, 1348, 393, 565, 8494, 336, 471, 29871, 29941, 29900, 448, 29871, 29941, 29945, 769, 4482, 881, 367, 29871, 29953, 29900, 29892, 541, 694, 29889, 1584, 411, 4236,

Only the response

In [7]:
for k, v in aspects_response.items():
    print(k)
    print(v)
    print('\n\n')

Gameplay

Gameplay:

* NA - No information provided in the review about gameplay.



Narrative

Narrative: NA



Accessibility

Accessibility: NA



Sound

Sound: NA



Graphics & Art Design

Graphics & Art Design:

* NA - The game's graphics and art design are not well-optimized and run at low framerates.



Performance

Sure! Here are the aspects of the game related to performance that can be extracted from the review:

1. Frame rate: Poorly optimized, runs between 25-35 FPS on both low and ultra settings.
2. Up-scaling: Disabling up-scaling improves the game's appearance, but it should not be necessary to do so.
3. Minimum frame rate: A $70 title should run at 50-60 FPS on the lowest settings minimum.
4. Optimization: Trying to optimize settings to get better frames has resulted in many hours of gameplay and no refund.

Here are the short descriptions for each aspect:

1. Frame rate: Poor performance with inconsistent frame rates.
2. Up-scaling: Unnecessary compromise in graphics qu

---

2-2-2-2-2 combination

In [8]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect '''

# aspects_response_22222 = {k: '' for k in GAME_ASPECTS}
sum_of_token_usage, sum_of_time_usage = 0, 0

for (start, end) in ((0, 2), (2, 4), (4, 6), (6, 8), (8, 10)):
    aspects = GAME_ASPECTS[start:end]
    my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    print(my_question)

    chain =  RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                template=prompt_template,
                input_variables=["summaries", "question"],
            )
        },
        return_source_documents=True,
    )

    response = chain.invoke(
        {
            'question': my_question
        }, config=chain_config
    )

    print('\'\'\'')
    print(response['answer'])
    print('\'\'\'')
    # aspects_response_22222[aspect] = response['answer']
    print('-'*10)
    print('Sources')
    for doc in response['source_documents']:
        print(doc.page_content)
        print(doc.metadata)
        print('-'*10)

    token_usage_obj = common_deque.pop()
    token_usage, time_usage = token_usage_obj['token_usage'], token_usage_obj['time_costed']
    sum_of_token_usage += token_usage
    sum_of_time_usage += time_usage
    print(f'Token Usage: {token_usage}')

    print('-'*20)

print('Summary of token usage:', sum_of_token_usage)
print('Summary of time usage:', sum_of_time_usage / 1e9, 's')

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect are : ['Gameplay', 'Narrative']


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their corresponding 20-word descriptions based on the review:\n\n['Gameplay']: Poorly optimized with inconsistent framerates (NA)\n\n['Narrative']: No mention of narrative quality or issues (NA)", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:16.085583694Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 29945, 448, 29871, 29941, 29945, 285, 567, 373, 1716, 4482, 322, 8494, 336, 6055, 29889, 366, 525, 270, 1348, 393, 565, 8494, 336, 471, 29871, 29941, 29900, 448, 29871, 29941, 29945, 769, 4482, 881, 367, 29871, 29953, 29900, 29892,

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their corresponding 20-word descriptions based on the review:\n\n['Accessibility']: Poorly optimized, runs at low frame rates. (NA)\n\n['Sound']: Game's sound quality is poor. (NA)", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:16.73655835Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 29945, 448, 29871, 29941, 29945, 285, 567, 373, 1716, 4482, 322, 8494, 336, 6055, 29889, 366, 525, 270, 1348, 393, 565, 8494, 336, 471, 29871, 29941, 29900, 448, 29871, 29941, 29945, 769, 4482, 881, 367, 29871, 29953, 29900, 29892, 541, 694, 2988

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their corresponding descriptions based on the review:\n\n['Graphics & Art Design']: Unclear, as the reviewer does not mention anything about the game's graphics or art design. NA\n\n['Performance']: Poorly optimized, runs between 25-35 FPS on both low and ultra settings. Even with max settings, the game still looked odd after disabling upscaling. A $70 title should run at 50-60 FPS on the lowest settings minimum. Trying to optimize settings resulted in many hours of frustration and no refund. Waiting for performance fix. NA", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:18.141722937Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their corresponding 20-word descriptions based on the review:\n\n['Bug']: Game has poor optimization, runs at 25-35 FPS on both low and ultra settings. (NA)\n\n['Suggestion']: A $70 title should run at 50-60 FPS on the lowest settings minimum. (NA)", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:19.681139423Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 29945, 448, 29871, 29941, 29945, 285, 567, 373, 1716, 4482, 322, 8494, 336, 6055, 29889, 366, 525, 270, 1348, 393, 565, 8494, 336, 471, 29871, 29941, 29900, 448, 29871, 29941, 2

---

3-3-4

In [9]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect '''

# aspects_response_334 = {k: '' for k in GAME_ASPECTS}
sum_of_token_usage, sum_of_time_usage = 0, 0

for (start, end) in ((0, 3), (3, 6), (6, 10)):
    aspects = GAME_ASPECTS[start:end]
    my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    print(my_question)

    chain =  RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                template=prompt_template,
                input_variables=["summaries", "question"],
            )
        },
        return_source_documents=True,
    )

    response = chain.invoke(
        {
            'question': my_question
        }, config=chain_config
    )

    print('\'\'\'')
    print(response['answer'])
    print('\'\'\'')
    # aspects_response_334[aspect] = response['answer']
    print('-'*10)
    print('Sources')
    for doc in response['source_documents']:
        print(doc.page_content)
        print(doc.metadata)
        print('-'*10)

    token_usage_obj = common_deque.pop()
    token_usage, time_usage = token_usage_obj['token_usage'], token_usage_obj['time_costed']
    sum_of_token_usage += token_usage
    sum_of_time_usage += time_usage
    print(f'Token Usage: {token_usage}')

    print('-'*20)

print('Summary of token usage:', sum_of_token_usage)
print('Summary of time usage:', sum_of_time_usage / 1e9, 's')

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect are : ['Gameplay', 'Narrative', 'Accessibility']


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their descriptions based on the review:\n\n['Gameplay']: Poorly optimized with inconsistent framerates (20 words)\n\n['Narrative']: None mentioned in the review (NA)\n\n['Accessibility']: No mention of accessibility features (NA)", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:21.167409248Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 29945, 448, 29871, 29941, 29945, 285, 567, 373, 1716, 4482, 322, 8494, 336, 6055, 29889, 366, 525, 270, 1348, 393, 565, 8494, 336, 471, 29871, 29941, 29900, 448, 29871, 29941, 29945, 769, 4482, 88

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their corresponding descriptions based on the review:\n\n['Sound']: Disappointing sound design with poor voice acting. NA\n\n['Graphics & Art Design']: Unimpressive graphics with lackluster art design. NA\n\n['Performance']: Poorly optimized, runs between 25-35 FPS on both low and ultra settings. Even with max settings, game still looked odd after disabling upscaling. A $70 title should run at 50-60 FPS on the lowest settings minimum. Trying to optimize settings resulted in many hours of frustration and a request for a refund. Waiting for performance issues to be fixed.", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T05:59:22.690187817Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 2

---

5 aspects in a prompt

In [11]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect '''

# aspects_response_55 = {k: '' for k in GAME_ASPECTS}
sum_of_token_usage, sum_of_time_usage = 0, 0

for (start, end) in ((0, 5), (5, 10)):
    aspects = GAME_ASPECTS[start:end]
    my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    print(my_question)

    chain =  RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                template=prompt_template,
                input_variables=["summaries", "question"],
            )
        },
        return_source_documents=True,
    )

    response = chain.invoke(
        {
            'question': my_question
        }, config=chain_config
    )

    print('\'\'\'')
    print(response['answer'])
    print('\'\'\'')
    # aspects_response_55[aspect] = response['answer']
    print('-'*10)
    print('Sources')
    for doc in response['source_documents']:
        print(doc.page_content)
        print(doc.metadata)
        print('-'*10)

    token_usage_obj = common_deque.pop()
    token_usage, time_usage = token_usage_obj['token_usage'], token_usage_obj['time_costed']
    sum_of_token_usage += token_usage
    sum_of_time_usage += time_usage
    print(f'Token Usage: {token_usage}')

    print('-'*20)

print('Summary of token usage:', sum_of_token_usage)
print('Summary of time usage:', sum_of_time_usage / 1e9, 's')

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect are : ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design']


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their corresponding descriptions based on the review:\n\n['Gameplay']: Disappointing experience with poor optimization. (20 words)\n\n['Narrative']: No clear information provided about the narrative. (NA)\n\n['Accessibility']: No mention of accessibility features. (NA)\n\n['Sound']: Poor sound quality and lack of immersion. (19 words)\n\n['Graphics & Art Design']: Graphics are poorly optimized and look odd. (20 words)", generation_info={'model': 'llama2:7b-chat-q4_K_M', 'created_at': '2024-07-09T06:04:42.008626669Z', 'response': '', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 3492, 526, 5183, 21804, 310, 263, 3748, 304, 2274, 278, 21862, 310, 278, 3748, 29889, 4803, 278, 1494, 12785, 310, 3030, 304, 1234, 1404, 29915, 29879, 1139, 29889, 29871, 13, 13, 3916, 29901, 6460, 368, 27545, 29892, 6057, 1546, 29871, 29906, 2994

---

10 aspects in a prompt

In [10]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect '''

# aspects_response_10 = {k: '' for k in GAME_ASPECTS}
sum_of_token_usage, sum_of_time_usage = 0, 0


aspects = GAME_ASPECTS
my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
print(my_question)

chain =  RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=prompt_template,
            input_variables=["summaries", "question"],
        )
    },
    return_source_documents=True,
)

response = chain.invoke(
    {
        'question': my_question
    }, config=chain_config
)

print('\'\'\'')
print(response['answer'])
print('\'\'\'')
# aspects_response_10[aspect] = response['answer']
print('-'*10)
print('Sources')
for doc in response['source_documents']:
    print(doc.page_content)
    print(doc.metadata)
    print('-'*10)

token_usage_obj = common_deque.pop()
token_usage, time_usage = token_usage_obj['token_usage'], token_usage_obj['time_costed']
sum_of_token_usage += token_usage
sum_of_time_usage += time_usage
print(f'Token Usage: {token_usage}')

print('-'*20)

print('Summary of token usage:', sum_of_token_usage)
print('Summary of time usage:', sum_of_time_usage / 1e9, 's')

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Extract the the following aspect of the game from the reviews. Output a short 20 words description for each aspect. The aspect are : ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']
Response in callback
generations=[[GenerationChunk(text="\nSure! Here are the aspects of the game and their corresponding 20-word descriptions based on the review:\n\n['Gameplay']: Disappointing experience with too much repetition. (NA)\n\n['Narrative']: Underwhelming story with too much repetition. (NA)\n\n['Accessibility']: No accessibility features mentioned in the review. (NA)\n\n['Sound']: Poor sound quality with little to no immersion. (NA)\n\n['Graphics & Art Design']: Unimpressive graphics with poor art design. (NA)\n\n['Performance']: Poorly optimized game with framerate issues. (60 fps minimum on lowest settings expected)\n\n['Bug']: No bug mentions in the review. (NA)\n\n['Suggestion']: Improve performance and opt

---

Observation

In short reviews, the quality of different prompting strategies have a few differences.

If prompting all aspects at a prompt -> little repetitive contents over different aspects. (Like the LLM assumes a new conservation for each call, if the aspects are similar yet appear in different prompts, similar result may reappear.)

The more the aspects prompted in a single prompt, the briefer the content of each aspect will be.

3-3-4 and 2-2-2-2-2 seems to be a decent balance between token usage, number of calls to the LLM.

For medium to long reviews where multiple points/ideas towards a single "aspect" is mentioned, the model may output things beyond our expected output.